<a href="https://colab.research.google.com/github/Rashmisharma0101/Personalized-whatsApp-chat---Text-generation-with-LSTM/blob/main/Text_Generation_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import re

In [9]:
cleaned_messages = []

with open('/content/WhatsApp Chat with Guddu Didi.txt', 'r', encoding = 'utf-8') as f:
    lines = f.readlines()

for line in lines:
    line = line.strip()

    # Skip system messages
    if "end-to-end encrypted" in line:
        continue

    # Skip media omitted
    if "<Media omitted>" in line:
        continue

    # WhatsApp format: "DATE, TIME - NAME: MESSAGE"
    if " - " in line and ":" in line:
        try:
            # Split at " - "
            after_date = line.split(" - ", 1)[1]

            # Split at first ":"
            name, message = after_date.split(":", 1)

            message = message.strip()
            if message:
                cleaned_messages.append(message)
        except:
            continue


In [11]:
cleaned_messages

['2nd effort at eyeliners',
 'Bhut sahi',
 'Mast',
 '😁😁',
 'Me to breakfast krne ja ro',
 'Subha subha ye ker rehe hai',
 'Kahan',
 'Bahar',
 'Haan bahar hu...number plate lagawane aaye hain',
 'Ok',
 'location: https://maps.google.com/?q=28.6542688,77.3439495',
 'Being Banarasi Brocade Printed Salwar Suit Material',
 'Pant k neeche k styles k liye',
 'Sahi hai',
 'https://www.facebook.com/2236394849932545/posts/2814854385419919/?sfnsn=wiwspmo',
 'Dekh behan',
 'Ye thoda sahi ho jaye fir aati hu',
 'Achha',
 'Achha lag ra h',
 'Haan accha aaya hai',
 'Back side esa hai',
 'Haan ye to araam se 2 sided bn jayega..agahe peeche wala to bn hi jayega',
 'Haan',
 'Wese hi bnao gi',
 'Sahi h',
 'Aayurda Indi Woven Kurta, Salwar & Dupatta Set',
 'SHREE MAYA BOUTIQUE Printed Kurta, Palazzo & Dupatta Set',
 'Achha h ye wala',
 'Haan',
 'Flipkart Amazon se mat mngao jada...ek number dungi use save kr lo...he had ship in Chandigarh...',
 'Achhe hain uske paas',
 'Mein nai mangwa rehe',
 'Tere liye 

In [18]:
data = " ".join(cleaned_messages)


In [20]:
data = re.sub(r'http\S+', '', data)

In [24]:
# clean emojis, URLS

data = re.sub(r'thttp\S+', '', data)
data = re.sub(r'[^\w\s]', '', data)
data = re.sub(r'\s+', ' ', data).strip()

In [25]:
data=  re.sub(r"<Media omitted>", "", data)
data = re.sub(r"\b\d{10,15}\b", "", data)
data = re.sub(r"\+?\d[\d\s-]{8,15}", "", data)

In [26]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
total_words = len(tokenizer.word_index) + 1

In [28]:
tokens = tokenizer.texts_to_sequences([data])[0]

seq_length = 10

sequences = []

for i in range(seq_length, len(tokens)):
  seq = tokens[i-seq_length:i+1]
  sequences.append(seq)

In [30]:
import numpy as np
import tensorflow as tf
sequences = np.array(sequences)

X = sequences[:, :-1]
y = sequences[:, -1]
y = tf.keras.utils.to_categorical(y, total_words)

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential([
    Embedding(total_words, 100, input_length=seq_length),
    LSTM(150),
    Dense(total_words, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [32]:
model.fit(X, y, epochs=50, batch_size=128)


Epoch 1/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 11s 68ms/step - loss: 7.1326
Epoch 2/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 8s 64ms/step - loss: 5.9204
Epoch 3/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 9s 71ms/step - loss: 5.7258
Epoch 4/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 9s 69ms/step - loss: 5.5657
Epoch 5/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 8s 63ms/step - loss: 5.5168
Epoch 6/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 9s 72ms/step - loss: 5.4313
Epoch 7/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 11s 76ms/step - loss: 5.3336
Epoch 8/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 8s 62ms/step - loss: 5.2026
Epoch 9/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 10s 78ms/step - loss: 5.1309
Epoch 10/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 10s 75ms/step - loss: 5.0228
Epoch 11/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 9s 73ms/step - loss: 4.8648
Epoch 12/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 8s 62ms/step - loss: 4.7578
Epoch 13/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 9s 73ms/step - loss: 4.6567
Epoch 14/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 9s 72ms/step - loss: 4.5199
Epoch 15/50
129/129 ━━━━━━━━━━━━━━━━━━━

In [33]:
import numpy as np

def generate_text(seed_text, next_words=20):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=seq_length, padding='pre')

        predicted = np.argmax(model.predict(token_list), axis=1)[0]

        for word, index in tokenizer.word_index.items():
            if index == predicted:
                seed_text += " " + word
                break

    return seed_text


In [35]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
print(generate_text("haan", 20))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
haan blood bhi bna ki highrisk h merinbhi dusri me chahe bhi dekhna ok ji nice del amazon ko ok ye
